### 深度卷积神经网络

#### 端到端

神经网络可以直接基于图像的原始像素进行分类，这种方法称为端到端（end-to-end）。

但是LeNet提出后的，我们依然还是用svm等机器学习来进行分类，并且效果不错。
    
    1、获取图像数据集；
    2、使用已有的特征提取函数生成图像的特征；
    3、使用机器学习模型对图像的特征分类。

但是神经网络派认为，特征也是可以学习的，而不需要人工去提取。不过由于数据缺乏和硬件计算能力不行，倒是深层的神经网络效果不行。

直到2012年的AlexNet。

#### AlexNet的模型结构

    AlexNet与LeNet的设计理念非常相似，但也有显著的区别。

    第一，与相对较小的LeNet相比，AlexNet包含8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。下面我们来详细描述这些层的设计。

    AlexNet第一层中的卷积窗口形状是11×11。因为ImageNet中绝大多数图像的高和宽均比MNIST图像的高和宽大10倍以上，ImageNet图像的物体占用更多的像素，所以需要更大的卷积窗口来捕获物体。第二层中的卷积窗口形状减小到5×5，之后全采用3×3。此外，第一、第二和第五个卷积层之后都使用了窗口形状为3×3、步幅为2的最大池化层。而且，AlexNet使用的卷积通道数也大于LeNet中的卷积通道数数十倍。

    紧接着最后一个卷积层的是两个输出个数为4096的全连接层。这两个巨大的全连接层带来将近1 GB的模型参数。由于早期显存的限制，最早的AlexNet使用双数据流的设计使一个GPU只需要处理一半模型。幸运的是，显存在过去几年得到了长足的发展，因此通常我们不再需要这样的特别设计了。

    第二，AlexNet将sigmoid激活函数改成了更加简单的ReLU激活函数。一方面，ReLU激活函数的计算更简单，例如它并没有sigmoid激活函数中的求幂运算。另一方面，ReLU激活函数在不同的参数初始化方法下使模型更容易训练。这是由于当sigmoid激活函数输出极接近0或1时，这些区域的梯度几乎为0，从而造成反向传播无法继续更新部分模型参数；而ReLU激活函数在正区间的梯度恒为1。因此，若模型参数初始化不当，sigmoid函数可能在正区间得到几乎为0的梯度，从而令模型无法得到有效训练。

    第三，AlexNet通过丢弃法（dropout）来控制全连接层的模型复杂度。而LeNet并没有使用丢弃法。

    第四，AlexNet引入了大量的图像增广，如翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合。

读取数据

    虽然论文中AlexNet使用ImageNet数据集，但因为ImageNet数据集训练时间较长，我们仍用前面的Fashion-MNIST数据集来演示AlexNet。读取数据的时候我们额外做了一步将图像高和宽扩大到AlexNet使用的图像高和宽224。这个可以通过Resize实例来实现。也就是说，我们在ToTensor实例前使用Resize实例，然后使用Compose实例来将这两个变换串联以方便调用。

#### AlexNet实现

In [1]:
#  basic package
import numpy as np
import keras
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# 导入数据
# fashion-mnist
def load_mnist(path, kind='train'):
    import os
    import gzip

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

X_train, y_train = load_mnist('F:/机器学习/动手学深度学习/data/fashion', kind='train')
X_test, y_test = load_mnist('F:/机器学习/动手学深度学习/data/fashion', kind='t10k')

In [3]:
X_train.shape

(60000, 784)

In [4]:
def image_resize(input, new_width, new_height):
    """
        imput:  ndarray
        return: ndarray
    """
    from PIL import Image
    import numpy as np
    img = Image.fromarray(input)
    new_img = img.resize((new_width, new_height),Image.ADAPTIVE)
    return np.array(new_img)

def show_imgage(im2):
    import matplotlib.pyplot as plt
    plt.imshow(im2)
    plt.axis('off')
    plt.show()

In [5]:
X_train[0].shape,np.array((X_train.shape[0], 224, 224))

((784,), array([60000,   224,   224]))

In [6]:
%%time
# 处理数据
from keras.utils.np_utils import to_categorical

sample_size = 1000

X_train_new = np.zeros([sample_size, 224, 224])

# 填充28 -224
for i in range(sample_size):
    X_train_new[i] = image_resize(X_train[i].reshape(28,28), 224, 224)

Wall time: 759 ms


In [7]:
%%time
X_train_1 = X_train_new / 255
x_train = X_train_1.reshape(-1, 224, 224, 1)

X_test_new = np.zeros([sample_size, 224, 224])
X_test_1 = X_test_new / 255
x_test = X_test_1.reshape(-1, 224, 224, 1)

y_train = to_categorical(y_train[:sample_size], num_classes=10)
y_test = to_categorical(y_test[:sample_size], num_classes=10)

Wall time: 575 ms


In [8]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((1000, 224, 224, 1), (1000, 224, 224, 1), (1000, 10), (1000, 10))

In [9]:
%%time
# 定义模型keras

batch_size = 32

model = Sequential([
    # part 1
    Conv2D(filters=96, kernel_size=(11, 11), activation='relu', strides=(4,4), padding='valid', input_shape=(224, 224, 1)), 
    MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid'),
    Conv2D(filters=256, kernel_size=(5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid'),
    
    # part 2
    # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
    # 前两个卷积层后不使用池化层来减小输入的高和宽
    Conv2D(filters=384, kernel_size=(3, 3), padding="same", activation='relu'),
    Conv2D(filters=384, kernel_size=(3, 3), padding="same", activation='relu'),
    Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid'),
    
    Flatten(),
    # 全连接层块
    Dense(4096,activation='relu'),
    Dropout(0.5),
    Dense(4096,activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# compile
sgd = SGD(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# conv0 output shape:      (1, 96, 54, 54)
# pool0 output shape:      (1, 96, 26, 26)
# conv1 output shape:      (1, 256, 26, 26)
# pool1 output shape:      (1, 256, 12, 12)
# conv2 output shape:      (1, 384, 12, 12)
# conv3 output shape:      (1, 384, 12, 12)
# conv4 output shape:      (1, 256, 12, 12)
# pool2 output shape:      (1, 256, 5, 5)
# dense0 output shape:     (1, 4096)
# dropout0 output shape:   (1, 4096)
# dense1 output shape:     (1, 4096)
# dropout1 output shape:   (1, 4096)
# dense2 output shape:     (1, 10)


Wall time: 205 ms


In [10]:
model.summary()

# for layer in model.layers:
#     print(layer.output_shape)
# for i in range(len(model.layers)):
#     print(model.get_layer(index=i).output)

# print("[INFO] Method 2...")
# for i in range(len(model.layers)):
#     print(model.get_layer(index=i).output)

# print("[INFO] Method 3...")
# for layer in model.layers:
#     print(layer.output_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 384)       885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 384)       1327488   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 256)       884992    
__________

In [11]:
# train
model.fit(x_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 55s 55ms/step - loss: 2.2979 - acc: 0.1270
Epoch 2/5
1000/1000 [==============================] - 14s 14ms/step - loss: 2.2802 - acc: 0.2220
Epoch 3/5
1000/1000 [==============================] - 14s 14ms/step - loss: 2.2197 - acc: 0.2580
Epoch 4/5
1000/1000 [==============================] - 14s 14ms/step - loss: 1.8851 - acc: 0.3290
Epoch 5/5
1000/1000 [==============================] - 14s 14ms/step - loss: 1.5763 - acc: 0.4090


In [12]:
# train
loss_and_metric = model.evaluate(x_test, y_test, batch_size=32)


1000/1000 [==============================] - 4s 4ms/step


In [13]:
print(loss_and_metric)

[2.3044380016326906, 0.087]
